In [1]:
import numpy as np
import pandas as pd
import random

In [2]:
# Important landmarks for Warrior II pose
IMPORTANT_LANDMARKS = [
    "nose", "left_eye", "right_eye",
    "left_shoulder", "right_shoulder",
    "left_elbow", "right_elbow",
    "left_index", "right_index",
    "left_hip", "right_hip",
    "left_knee", "right_knee",
    "left_heel", "right_heel",
    "left_foot_index", "right_foot_index"
]


# Important Functions

In [6]:

def get_landmark_columns(landmark_name):
    """Get all column names for a landmark (x, y, z, visibility)"""
    return [
        f'{landmark_name}_x',
        f'{landmark_name}_y',
        f'{landmark_name}_z',
        f'{landmark_name}_v'
    ]


def modify_landmark(landmarks, landmark_name, x_delta=0, y_delta=0, z_delta=0):
    """Helper function to modify a specific landmark"""
    modified = landmarks.copy()
    modified[f'{landmark_name}_x'] += x_delta
    modified[f'{landmark_name}_y'] += y_delta
    modified[f'{landmark_name}_z'] += z_delta
    return modified


def create_bent_front_knee(landmarks):
    """Front knee bends too much (should be at 90 degrees)"""
    modified = landmarks.copy()
    
    # Bend the front (left) knee more - knee drops down
    bend_amount = np.random.uniform(0.05, 0.15)
    modified = modify_landmark(modified, 'left_knee', y_delta=-bend_amount)
    modified = modify_landmark(modified, 'left_hip', y_delta=-bend_amount * 0.6)
    
    return modified


def create_arms_dropped(landmarks):
    """Arms not parallel to ground (one or both dropped)"""
    modified = landmarks.copy()
    
    # Drop left or right arm
    if random.random() < 0.5:
        # Drop left arm
        drop_amount = np.random.uniform(0.1, 0.25)
        modified = modify_landmark(modified, 'left_elbow', y_delta=drop_amount)
        modified = modify_landmark(modified, 'left_index', y_delta=drop_amount)
    else:
        # Drop right arm
        drop_amount = np.random.uniform(0.1, 0.25)
        modified = modify_landmark(modified, 'right_elbow', y_delta=drop_amount)
        modified = modify_landmark(modified, 'right_index', y_delta=drop_amount)
    
    return modified


def create_hips_rotated(landmarks):
    """Hips not squared/facing forward"""
    modified = landmarks.copy()
    
    # Rotate hips - one forward, one back
    rotation = np.random.uniform(0.05, 0.15)
    modified = modify_landmark(modified, 'left_hip', x_delta=rotation)
    modified = modify_landmark(modified, 'right_hip', x_delta=-rotation)
    
    # Shoulders follow hip rotation slightly
    modified = modify_landmark(modified, 'left_shoulder', x_delta=rotation * 0.6)
    modified = modify_landmark(modified, 'right_shoulder', x_delta=-rotation * 0.6)
    
    return modified


def create_back_foot_wrong_angle(landmarks):
    """Back foot not at proper angle (should be ~90 degrees)"""
    modified = landmarks.copy()
    
    # Change back foot angle
    angle_change = np.random.uniform(0.08, 0.18)
    modified = modify_landmark(modified, 'right_foot_index', x_delta=angle_change)
    modified = modify_landmark(modified, 'right_heel', x_delta=angle_change * 0.8)
    
    return modified


def create_leaning_forward(landmarks):
    """Torso leaning forward instead of upright"""
    modified = landmarks.copy()
    
    # Lean torso forward
    lean_amount = np.random.uniform(0.05, 0.15)
    forward_shift = np.random.uniform(0.02, 0.08)
    
    # Upper body leans forward
    modified = modify_landmark(modified, 'nose', y_delta=lean_amount, x_delta=forward_shift)
    modified = modify_landmark(modified, 'left_eye', y_delta=lean_amount, x_delta=forward_shift)
    modified = modify_landmark(modified, 'right_eye', y_delta=lean_amount, x_delta=forward_shift)
    modified = modify_landmark(modified, 'left_shoulder', y_delta=lean_amount * 0.8, x_delta=forward_shift * 0.7)
    modified = modify_landmark(modified, 'right_shoulder', y_delta=lean_amount * 0.8, x_delta=forward_shift * 0.7)
    
    return modified


def create_narrow_stance(landmarks):
    """Feet too close together (stance not wide enough)"""
    modified = landmarks.copy()
    
    # Bring right foot closer to left
    narrow_amount = np.random.uniform(0.1, 0.2)
    
    modified = modify_landmark(modified, 'right_heel', x_delta=narrow_amount)
    modified = modify_landmark(modified, 'right_foot_index', x_delta=narrow_amount)
    modified = modify_landmark(modified, 'right_knee', x_delta=narrow_amount * 0.7)
    modified = modify_landmark(modified, 'right_hip', x_delta=narrow_amount * 0.3)
    
    return modified


def create_shoulders_not_aligned(landmarks):
    """Shoulders not aligned (tilted)"""
    modified = landmarks.copy()
    
    # Tilt shoulders
    tilt = np.random.uniform(0.05, 0.12)
    modified = modify_landmark(modified, 'left_shoulder', y_delta=tilt)
    modified = modify_landmark(modified, 'right_shoulder', y_delta=-tilt)
    
    # Corresponding elbow adjustments
    modified = modify_landmark(modified, 'left_elbow', y_delta=tilt * 0.5)
    modified = modify_landmark(modified, 'right_elbow', y_delta=-tilt * 0.5)
    
    return modified


def create_arms_bent(landmarks):
    """Arms bent at elbows instead of straight"""
    modified = landmarks.copy()
    
    # Bend one or both arms
    bend_amount = np.random.uniform(0.08, 0.15)
    
    if random.random() < 0.5:
        # Bend left arm
        modified = modify_landmark(modified, 'left_elbow', y_delta=bend_amount, x_delta=bend_amount * 0.5)
        modified = modify_landmark(modified, 'left_index', y_delta=bend_amount * 0.7)
    else:
        # Bend right arm
        modified = modify_landmark(modified, 'right_elbow', y_delta=bend_amount, x_delta=-bend_amount * 0.5)
        modified = modify_landmark(modified, 'right_index', y_delta=bend_amount * 0.7)
    
    return modified



# Main function to create augumented dataset

In [7]:

def augment_warrior_ii_dataset(input_csv, output_csv, samples_per_class=50):
    """
    Create augmented dataset with multiple incorrect pose classes
    Works specifically with 17 important landmarks
    
    Args:
        input_csv: Path to CSV with correct Warrior II poses (filtered to 17 landmarks)
        output_csv: Path to save augmented dataset
        samples_per_class: Number of samples to generate for each incorrect class
    """
    print(f"📊 Loading correct poses from {input_csv}...")
    df = pd.read_csv(input_csv)
    
    # Verify we have the right landmarks
    expected_cols = ['label']
    for lm in IMPORTANT_LANDMARKS:
        expected_cols.extend(get_landmark_columns(lm))
    
    missing_cols = [col for col in expected_cols if col not in df.columns]
    if missing_cols:
        print(f"⚠️  Warning: Missing columns: {missing_cols[:5]}...")
        print(f"   Make sure your CSV has been filtered to the 17 important landmarks")
    
    # Filter only correct poses
    df_correct = df[df['label'] == 'c'].copy()
    print(f"✅ Found {len(df_correct)} correct pose samples")
    print(f"📏 Using {len([col for col in df.columns if col != 'label']) // 4} landmarks")
    
    # Define augmentation functions with their labels
    augmentations = {
        'bent_front_knee': create_bent_front_knee,
        'arms_dropped': create_arms_dropped,
        'arms_bent': create_arms_bent,
        'hips_rotated': create_hips_rotated,
        'back_foot_wrong_angle': create_back_foot_wrong_angle,
        'leaning_forward': create_leaning_forward,
        'narrow_stance': create_narrow_stance,
        'shoulders_not_aligned': create_shoulders_not_aligned
    }
    
    # Store all augmented samples
    all_samples = [df_correct]
    
    # Generate augmented samples for each class
    for label, aug_func in augmentations.items():
        print(f"🔄 Generating {samples_per_class} samples for class: {label}")
        augmented_samples = []
        
        for _ in range(samples_per_class):
            # Randomly select a correct pose
            correct_sample = df_correct.sample(n=1).iloc[0].copy()
            
            # Apply augmentation (exclude 'label' column)
            landmark_cols = [col for col in correct_sample.index if col != 'label']
            landmarks = correct_sample[landmark_cols]
            
            # Apply the augmentation function
            modified_landmarks = aug_func(landmarks)
            modified_landmarks['label'] = label
            
            augmented_samples.append(modified_landmarks)
        
        # Convert to dataframe
        df_aug = pd.DataFrame(augmented_samples)
        all_samples.append(df_aug)
        print(f"   ✓ Created {len(df_aug)} samples")
    
    # Combine all samples
    df_final = pd.concat(all_samples, ignore_index=True)
    
    # Shuffle the dataset
    df_final = df_final.sample(frac=1, random_state=42).reset_index(drop=True)
    
    # Save to CSV
    df_final.to_csv(output_csv, index=False)
    
    print(f"\n✅ Augmented dataset saved to {output_csv}")
    print(f"\n📈 Class Distribution:")
    class_counts = df_final['label'].value_counts().sort_index()
    for label, count in class_counts.items():
        print(f"   {label:25s}: {count:4d} samples")
    print(f"\n{'='*50}")
    print(f"Total samples: {len(df_final)}")
    print(f"Total classes: {df_final['label'].nunique()}")
    print(f"Features per sample: {len([col for col in df_final.columns if col != 'label'])}")
    
    return df_final

In [ ]:
# df = pd.read_csv("data_csv/train.csv")
# print(len(df))

420


In [8]:
if __name__ == "__main__":
    # Generate augmented dataset
    df = pd.read_csv("train.csv")

    no_of_data_per_class = len(df)
    df_augmented = augment_warrior_ii_dataset(
        input_csv='train.csv',      # filtered CSV 
        output_csv='train_augmented.csv',    # Output with all classes
        samples_per_class = no_of_data_per_class                  # Adjust based on your needs
    )
        
    print("\n🎯 Dataset ready for training!")

📊 Loading correct poses from train.csv...
✅ Found 420 correct pose samples
📏 Using 17 landmarks
🔄 Generating 420 samples for class: bent_front_knee
   ✓ Created 420 samples
🔄 Generating 420 samples for class: arms_dropped
   ✓ Created 420 samples
🔄 Generating 420 samples for class: arms_bent
   ✓ Created 420 samples
🔄 Generating 420 samples for class: hips_rotated
   ✓ Created 420 samples
🔄 Generating 420 samples for class: back_foot_wrong_angle
   ✓ Created 420 samples
🔄 Generating 420 samples for class: leaning_forward
   ✓ Created 420 samples
🔄 Generating 420 samples for class: narrow_stance
   ✓ Created 420 samples
🔄 Generating 420 samples for class: shoulders_not_aligned
   ✓ Created 420 samples

✅ Augmented dataset saved to train_augmented.csv

📈 Class Distribution:
   arms_bent                :  420 samples
   arms_dropped             :  420 samples
   back_foot_wrong_angle    :  420 samples
   bent_front_knee          :  420 samples
   c                        :  420 samples
  